In [25]:
import numbers
from html import escape
from decimal import Decimal

In [26]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

def html_real(a):
    return f'{round(a, 2):.2f}'

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l):
    items = (f'<li>{htmlize(item)}</li>' for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = (f'<li>{k}={v}</li>' for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [27]:
def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    else:
        return html_escape(arg)

<ul>
<li>Py<br/>
Tho<br/>
Pet is &lt; 100</li>
<li><ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
<li>3(<i>0x3</i>)</li>
</ul></li>
<li>100(<i>0x64</i>)</li>
</ul>


In [31]:
del htmlize

In [40]:
def htmlize(arg):
    register = {
        object: html_escape,
        int: html_int,
        float: html_real,
        Decimal: html_real,
        list: html_list,
        tuple: html_list,
        set: html_list,
        dict: html_dict,
        str: html_str
    }
    return register.get(type(arg), html_escape)(arg)

In [39]:
print(htmlize(10+10j))

(10+10j)


In [41]:
def singledispatch(fn):
    registry = {}
    registry[object] = fn
    def inner(arg):
        return registry[object](arg)
    return inner

In [42]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [44]:
htmlize('1 < 10')

'1 &lt; 10'

In [65]:
def singledispatch(fn):
    registry = {}
    registry[object] = fn
    
    def decorated(arg):
        return registry.get(type(arg), registry[object])(arg)
    
    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner
    
    def dispatch(type_):
        return registry.get(type_, registry[object])
    
    # decorated.registry = registry
    decorated.register = register
    decorated.dispatch = dispatch
    
    return decorated

In [66]:
@singledispatch
def htmlize(arg):
    return escape(str(arg))

In [67]:
@htmlize.register(int)
def html_int(arg):
    return f'{arg}(<i>{str(hex(arg))}</i>)'

In [68]:
htmlize(123)

'123(<i>0x7b</i>)'

In [69]:
htmlize.dispatch(int)

<function __main__.html_int(a)>

In [70]:
htmlize.dispatch(str)

<function __main__.htmlize(arg)>

In [71]:
from numbers import Integral

In [72]:
class Person:
    pass

In [86]:
class Student(Person):
    pass

In [87]:
student1 = Student()

In [88]:
isinstance(False, int)

True

In [93]:
Student.__mro__[-2]

__main__.Person

In [94]:
from functools import singledispatch

In [97]:
from numbers import Integral
from collections.abc import Sequence
from html import escape

In [120]:
@singledispatch
def htmlize(arg):
    return escape(str(arg))

In [121]:
htmlize.__dict__

{'register': <function functools.singledispatch.<locals>.register(cls, func=None)>,
 'dispatch': <function functools.singledispatch.<locals>.dispatch(cls)>,
 'registry': mappingproxy({object: <function __main__.htmlize(arg)>}),
 '_clear_cache': <bound method MutableMapping.clear of <WeakKeyDictionary at 0x7fb6e75a86e0>>,
 '__wrapped__': <function __main__.htmlize(arg)>}

In [122]:
htmlize.dispatch(str)

<function __main__.htmlize(arg)>

In [124]:
@htmlize.register(Integral)
def htmlize_integral_number(arg):
   return f'{arg}(<i>{str(hex(arg))}</i>)' 

In [125]:
htmlize(123)

'123(<i>0x7b</i>)'

In [126]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize(arg)>,
              numbers.Integral: <function __main__.htmlize_integral_number(arg)>})

In [127]:
@htmlize.register(Sequence)
def htmlize_sequence(arg):
    items = (f'<li>{htmlize(item)}</li>' for item in arg)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [129]:
print(htmlize([1, 2, 3]))

<ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
<li>3(<i>0x3</i>)</li>
</ul>


In [131]:
@htmlize.register(set)
def htmlize_sequence(arg):
    items = (f'<li>{htmlize(item)}</li>' for item in arg)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [132]:
print(htmlize([(1, 2, 3), {True, False}]))

<ul>
<li><ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
<li>3(<i>0x3</i>)</li>
</ul></li>
<li><ul>
<li>False(<i>0x0</i>)</li>
<li>True(<i>0x1</i>)</li>
</ul></li>
</ul>


In [133]:
htmlize('python')

RecursionError: maximum recursion depth exceeded

In [134]:
@htmlize.register(str)
def htmlize_str(arg):
    return html_escape(arg).replace('\n', '<br/>\n')

In [136]:
htmlize('python')

'python'

In [138]:
@singledispatch
def htmlize(arg):
    return escape(str(arg))

In [143]:
@htmlize.register(set)
def _(arg):
   return f'{arg}(<i>{str(hex(arg))}</i>)' 

In [147]:
htmlize.registry[set] == htmlize.registry[Integral]

False

In [148]:
lst = []
a = 10
lst.append(a)
a = True
lst.append(a)
del a

In [149]:
lst[0], lst[1]

(10, True)

In [153]:
id(htmlize.dispatch(Integral)), id(htmlize.dispatch(set)) 

(140423204430816, 140423204430496)